# Spark API Exercises

## Testing Installation

In [5]:
import pandas as pd
import numpy as np

In [1]:
from pyspark.sql.functions import *

In [2]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
spark.range(5).show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/09 10:41:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



In [3]:
import pyspark

spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [4]:
import multiprocessing
import pyspark

nprocs = multiprocessing.cpu_count()

spark = (pyspark.sql.SparkSession.builder
 .master('local')
 .config('spark.jars.packages', 'mysql:mysql-connector-java:8.0.16')
 .config('spark.driver.memory', '4G')
 .config('spark.driver.cores', nprocs)
 .config('spark.sql.shuffle.partitions', nprocs)
 .appName('MySparkApplication')
 .getOrCreate())

22/09/09 10:53:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## 1. Create a spark data frame that contains your favorite programming languages.

- a. The name of the column should be language
- b. View the schema of the dataframe
- c. Output the shape of the dataframe
- d. Show the first 5 records in the dataframe

In [9]:
# Creating dataframe with my favorite languages
pandas_dataframe = pd.DataFrame({'Python', 'SQL', 'HTML', 'CSS', 'JavaScript', 'R', 'Markdown'}, columns = ['languages'])

In [10]:
pandas_dataframe

,languages
0,R
1,HTML
2,Markdown
3,JavaScript
4,CSS
5,SQL
6,Python


In [11]:
# Convert pandas dataframe to spark dataframe
language = spark.createDataFrame(pandas_dataframe)
language

DataFrame[languages: string]

In [12]:
# Viewing the top 5 records
language.show(5)

+----------+
| languages|
+----------+
|         R|
|      HTML|
|  Markdown|
|JavaScript|
|       CSS|
+----------+
only showing top 5 rows



In [14]:
language.describe().show()

+-------+---------+
|summary|languages|
+-------+---------+
|  count|        7|
|   mean|     null|
| stddev|     null|
|    min|      CSS|
|    max|      SQL|
+-------+---------+

